In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import os
import numpy as np
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re

### Read Data

In [2]:
df = pd.read_csv(r"C:\\Users\\joana\\Documents\\GitHub\\CSCI6405\\src\\data\\Reddit_MentalHealth_250.csv")
df.head()

,id,title,author,author_id,author_karma,author_flair,upvotes,score,upvote_ratio,post_ts,...,comments,nsfw,self_text,spoiler,stickied,subreddit,subreddit_id,subreddit_subs,url,category
0,18qa8qj,I’m depressed and anxious this Christmas,mrsunsfan,8kdawssf,320425.0,NaN,0,0,0.49,1.703474e+09,...,29,False,I’ve been struggling since my ex broke up with...,False,False,Anxiety,2qmij,661128,https://www.reddit.com/r/Anxiety/comments/18qa...,Controversial
1,120xdga,Today i was at a neurologist and i told him th...,useralreadyused,g3wz23b7,144.0,NaN,0,0,0.49,1.679690e+09,...,85,False,NaN,False,False,Anxiety,2qmij,661128,https://www.reddit.com/r/Anxiety/comments/120x...,Controversial
2,1902yq0,Do you watch porn?,Admirable_Strike4114,cpd2lm0x,1836.0,NaN,3,3,0.54,1.704557e+09,...,22,False,Pornography is the worst factor that aggravate...,False,False,Anxiety,2qmij,661128,https://www.reddit.com/r/Anxiety/comments/1902...,Controversial
3,roksot,Welp guess I’ll die a virgin,ThrowawayT0,NaN,-inf,NaN,1,1,0.52,1.640478e+09,...,52,False,\nLady who I thought was flirting well... she ...,False,False,Anxiety,2qmij,661128,https://www.reddit.com/r/Anxiety/comments/roks...,Controversial
4,2kvv6z,[Rant] So today I saw a guy in town dressed a ...,tumblingturtle,in9cu,15.0,NaN,2,2,0.52,1.414764e+09,...,17,False,"So there I was walking through town, noticed s...",False,False,Anxiety,2qmij,661128,https://www.reddit.com/r/Anxiety/comments/2kvv...,Controversial


In [3]:
df = df[df['subreddit'] != 'selfimprovement'].reset_index(drop=True)

In [4]:
df.subreddit.unique()

array(['Anxiety', 'AnxietyDepression', 'depression', 'depression_help',
       'mentalhealth', 'MentalHealthSupport', 'mentalillness', 'ptsd',
       'selfhelp', 'socialanxiety'], dtype=object)

In [5]:
i = 999
df[['subreddit', 'subreddit_subs']].iloc[[i, i + 1000, i + 1000*2, i + 1000*3, 
                                          i + 1000*4, i + 1000*5, i + 1000*6, 
                                          i + 1000*7, i + 1000*8, i + 1000*9]]

,subreddit,subreddit_subs
999,Anxiety,661109
1999,AnxietyDepression,55678
2999,depression,1007636
3999,depression_help,93515
4999,mentalhealth,443558
5999,MentalHealthSupport,41518
6999,mentalillness,149332
7999,ptsd,100210
8999,selfhelp,172414
9999,socialanxiety,401691


### Time analysis

In [7]:
df['post_ts'] = pd.to_datetime(df['post_ts'], unit = 's')
df.head()

,id,title,author,author_id,author_karma,author_flair,upvotes,score,upvote_ratio,post_ts,...,comments,nsfw,self_text,spoiler,stickied,subreddit,subreddit_id,subreddit_subs,url,category
0,18qa8qj,I’m depressed and anxious this Christmas,mrsunsfan,8kdawssf,320425.0,NaN,0,0,0.49,2023-12-25 03:17:52,...,29,False,I’ve been struggling since my ex broke up with...,False,False,Anxiety,2qmij,661128,https://www.reddit.com/r/Anxiety/comments/18qa...,Controversial
1,120xdga,Today i was at a neurologist and i told him th...,useralreadyused,g3wz23b7,144.0,NaN,0,0,0.49,2023-03-24 20:28:54,...,85,False,NaN,False,False,Anxiety,2qmij,661128,https://www.reddit.com/r/Anxiety/comments/120x...,Controversial
2,1902yq0,Do you watch porn?,Admirable_Strike4114,cpd2lm0x,1836.0,NaN,3,3,0.54,2024-01-06 15:56:43,...,22,False,Pornography is the worst factor that aggravate...,False,False,Anxiety,2qmij,661128,https://www.reddit.com/r/Anxiety/comments/1902...,Controversial
3,roksot,Welp guess I’ll die a virgin,ThrowawayT0,NaN,-inf,NaN,1,1,0.52,2021-12-26 00:11:57,...,52,False,\nLady who I thought was flirting well... she ...,False,False,Anxiety,2qmij,661128,https://www.reddit.com/r/Anxiety/comments/roks...,Controversial
4,2kvv6z,[Rant] So today I saw a guy in town dressed a ...,tumblingturtle,in9cu,15.0,NaN,2,2,0.52,2014-10-31 14:04:24,...,17,False,"So there I was walking through town, noticed s...",False,False,Anxiety,2qmij,661128,https://www.reddit.com/r/Anxiety/comments/2kvv...,Controversial


In [8]:
min(df['post_ts'])

Timestamp('2009-01-16 23:57:26')

In [9]:
max(df['post_ts'])

Timestamp('2024-02-16 20:00:49')

### Karma span

In [10]:
min(df['author_karma'])

-inf

In [11]:
max(df['author_karma'])

769451.0

### Upvotes/Score span

In [12]:
(df['upvotes'] == df['score']).value_counts()

True    10000
dtype: int64

In [13]:
min(df['upvotes'])

0

In [14]:
max(df['upvotes'])

27939

### Upvotes ratio

In [15]:
min(df['upvote_ratio'])

0.13

In [16]:
max(df['upvote_ratio'])

1.0

### Comments span

In [17]:
min(df['comments'])

0

In [18]:
max(df['comments'])

18657

### Count per column

In [19]:
df[['subreddit']].groupby(by = ['subreddit']).value_counts()

subreddit
Anxiety                1000
AnxietyDepression      1000
MentalHealthSupport    1000
depression             1000
depression_help        1000
mentalhealth           1000
mentalillness          1000
ptsd                   1000
selfhelp               1000
socialanxiety          1000
dtype: int64

In [20]:
df[['nsfw']].groupby(by = ['nsfw']).value_counts()

nsfw
False    9637
True      363
dtype: int64

In [21]:
df[['subreddit', 'nsfw']].groupby(by = ['nsfw']).value_counts()

nsfw   subreddit          
False  selfhelp               987
       socialanxiety          986
       AnxietyDepression      979
       Anxiety                977
       depression             966
       MentalHealthSupport    962
       depression_help        960
       mentalhealth           954
       mentalillness          952
       ptsd                   914
True   ptsd                    86
       mentalillness           48
       mentalhealth            46
       depression_help         40
       MentalHealthSupport     38
       depression              34
       Anxiety                 23
       AnxietyDepression       21
       socialanxiety           14
       selfhelp                13
dtype: int64

In [22]:
df[['spoiler']].groupby(by = ['spoiler']).value_counts()

spoiler
False    9949
True       51
dtype: int64

In [23]:
df[['subreddit', 'spoiler']].groupby(by = ['spoiler']).value_counts()

spoiler  subreddit          
False    depression             1000
         mentalhealth           1000
         selfhelp               1000
         socialanxiety          1000
         Anxiety                 997
         AnxietyDepression       995
         depression_help         994
         ptsd                    991
         MentalHealthSupport     990
         mentalillness           982
True     mentalillness            18
         MentalHealthSupport      10
         ptsd                      9
         depression_help           6
         AnxietyDepression         5
         Anxiety                   3
dtype: int64

In [24]:
df[['stickied']].groupby(by = ['stickied']).value_counts()

stickied
False    9986
True       14
dtype: int64

In [25]:
df[['subreddit', 'stickied']].groupby(by = ['stickied']).value_counts()

stickied  subreddit          
False     selfhelp               1000
          mentalillness          1000
          MentalHealthSupport    1000
          socialanxiety           999
          AnxietyDepression       999
          ptsd                    998
          Anxiety                 998
          mentalhealth            998
          depression_help         997
          depression              997
True      depression                3
          depression_help           3
          Anxiety                   2
          mentalhealth              2
          ptsd                      2
          AnxietyDepression         1
          socialanxiety             1
dtype: int64

In [26]:
df[['locked']].groupby(by = ['locked']).value_counts()

locked
False    9970
True       30
dtype: int64

In [29]:
df.dtypes

id                 object
title              object
author             object
author_id          object
author_karma      float64
author_flair       object
upvotes             int64
score               int64
upvote_ratio      float64
post_ts           float64
distinguished      object
edited             object
original             bool
self                 bool
locked               bool
name               object
comments            int64
nsfw                 bool
self_text          object
spoiler              bool
stickied             bool
subreddit          object
subreddit_id       object
subreddit_subs      int64
url                object
category           object
dtype: object

In [36]:
df[['edited']].groupby(by = ['edited']).value_counts()

edited
1307972177.0       1
1327417235.0       1
1327475624.0       1
1336368037.0       1
1345001519.0       1
                ... 
1708097643.0       2
1708097977.0       2
1708098705.0       2
1708099048.0       2
False           9934
Length: 905, dtype: int64

In [27]:
df[['subreddit', 'locked']].groupby(by = ['locked']).value_counts()

locked  subreddit          
False   AnxietyDepression      1000
        MentalHealthSupport    1000
        depression             1000
        mentalillness          1000
        selfhelp               1000
        socialanxiety           998
        depression_help         996
        ptsd                    995
        Anxiety                 994
        mentalhealth            987
True    mentalhealth             13
        Anxiety                   6
        ptsd                      5
        depression_help           4
        socialanxiety             2
dtype: int64

In [20]:
df[['self']].groupby(by = ['self']).value_counts()

self
False    1011
True     9989
dtype: int64

In [29]:
df[['subreddit', 'self']].groupby(by = ['self']).value_counts()

self   subreddit          
False  socialanxiety           296
       depression_help         285
       mentalillness           146
       mentalhealth             82
       Anxiety                  73
       selfhelp                 62
       AnxietyDepression        27
       depression               24
       ptsd                      7
True   MentalHealthSupport    1000
       ptsd                    993
       depression              976
       AnxietyDepression       973
       selfhelp                938
       Anxiety                 927
       mentalhealth            918
       mentalillness           854
       depression_help         715
       socialanxiety           704
dtype: int64

In [30]:
df[['original']].groupby(by = ['original']).value_counts()

original
False    10000
dtype: int64

In [31]:
df[['distinguished']].groupby(by = ['distinguished']).value_counts()

distinguished
moderator    11
dtype: int64

In [32]:
df[['subreddit', 'distinguished']].groupby(by = ['distinguished']).value_counts()

distinguished  subreddit          
moderator      depression             4
               ptsd                   3
               AnxietyDepression      1
               MentalHealthSupport    1
               depression_help        1
               mentalhealth           1
dtype: int64

### NA count

In [33]:
df.isna().sum()

id                   0
title                0
author             784
author_id         1134
author_karma         0
author_flair      9958
upvotes              0
score                0
upvote_ratio         0
post_ts              0
distinguished     9989
edited               0
original             0
self                 0
locked               0
name                 0
comments             0
nsfw                 0
self_text         1352
spoiler              0
stickied             0
subreddit            0
subreddit_id         0
subreddit_subs       0
url                  0
category             0
dtype: int64

### No text body content

In [43]:
df[df['self_text'].isna()][['title', 'url', 'self', 'self_text']]

,title,url,self,self_text
1,Today i was at a neurologist and i told him th...,https://www.reddit.com/r/Anxiety/comments/120x...,True,NaN
7,Blew my mind.,https://fbcdn-sphotos-b-a.akamaihd.net/hphotos...,False,NaN
22,F-E-A-R,http://i.imgur.com/MCmofQP.jpg,False,NaN
39,The Onion is here for you,http://www.theonion.com/articles/still-some-nu...,False,NaN
40,"I’m prescribed .5mg of Xanax, as needed but no...",http://anxietyadventures.wordpress.com/2012/09...,False,NaN
...,...,...,...,...
9995,My social battery is zero before going out,https://i.redd.it/6940lvlpy7b51.png,False,NaN
9996,Everyday at college,https://i.redd.it/yx3cfdwwjni31.jpg,False,NaN
9997,I can’t even talk normally,https://i.redd.it/k2z8dgbx1yz51.jpg,False,NaN
9998,Everyday struggle 😔,https://i.redd.it/c5pcwl8lu3c61.jpg,False,NaN


In [45]:
df[df['self_text'].isna()]['url'].iloc[0]

'https://www.reddit.com/r/Anxiety/comments/120xdga/today_i_was_at_a_neurologist_and_i_told_him_that/'

In [47]:
df[df['self_text'].isna()]['url'].iloc[2]

'http://i.imgur.com/MCmofQP.jpg'

### NSFW analysis

In [13]:
# Source https://gist.github.com/sebleier/554280?permalink_comment_id=3126707#gistcomment-3126707
custom_stopwords = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"]

In [14]:
# Text simplification
def preprocess_text(text):
    # Remove emoticons, symbols, etc.
	# source https://medium.com/codex/making-wordcloud-of-tweets-using-python-ca114b7a4ef4
	regex_pattern = re.compile(pattern = "["
			u"\U0001F600-\U0001F64F"  # emoticons
			u"\U0001F300-\U0001F5FF"  # symbols & pictographs
			u"\U0001F680-\U0001F6FF"  # transport & map symbols
			u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
							"]+", flags = re.UNICODE)
	text = re.sub(regex_pattern, '', text)

	text = text.encode('ascii','ignore') # remove zero-width-space x200b
	text = text.decode('utf-8')	

	# Lowercase
	text = text.lower()
	# Remove punctuation
	text = re.sub(r'[^\w\s]', '', text)
	# Tokenize words
	tokens = nltk.word_tokenize(text)
	# Remove stopwords (default and custom)
	stopwords_set = set(stopwords.words('english'))
	stopwords_set.update(custom_stopwords)
    # Filter tokens
	new_tokens = [token for token in tokens if token not in stopwords_set]
	# Stemming
	porter_stemmer = PorterStemmer()
	tokens_after_stemming = [porter_stemmer.stem(token) for token in new_tokens]

	return ' '.join(tokens_after_stemming)

In [15]:
df['self_text'] = df['self_text'].astype(object).replace(np.nan, '')
df_nsfw_text = df[df['nsfw'] == True]['title'] + ' ' + df[(df['nsfw'] == True)]['self_text']
df_nsfw_text = df_nsfw_text.astype(object).replace(np.nan, '')
preproc_text = []
for sample in df_nsfw_text:
	preproc_text.append(preprocess_text(sample))
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(preproc_text)
WORDCLOUD_DIRECTORY = 'C:\\Users\\joana\\Documents\\GitHub\\CSCI6405\\src\\data'

combined_text = " ".join(preproc_text)  
words = combined_text.lower().split()
word_freq = Counter(words)

wordcloud = WordCloud(width = 800, height = 400, background_color = 'white').generate_from_frequencies(word_freq)  
output_filename = os.path.join(WORDCLOUD_DIRECTORY, f'nsfw_word_cloud3.png')
wordcloud.to_file(output_filename)

### All post text analysis

In [16]:
df['self_text'] = df['self_text'].astype(object).replace(np.nan, '')
df_text = df['title'] + ' ' + df['self_text']
preproc_text = []
for sample in df_text:
	preproc_text.append(preprocess_text(sample))
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(preproc_text)
WORDCLOUD_DIRECTORY = 'C:\\Users\\joana\\Documents\\GitHub\\CSCI6405\\src\\data'

combined_text = " ".join(preproc_text)  
words = combined_text.lower().split()
word_freq = Counter(words)

wordcloud = WordCloud(width = 800, height = 400, background_color = 'white').generate_from_frequencies(word_freq)  
output_filename = os.path.join(WORDCLOUD_DIRECTORY, f'all_word_cloud3.png')
wordcloud.to_file(output_filename)